# Assignment 5: Construction Equipment Detection using YOLOv5

### Overview
In this assignment, you will build an object detection model to detect construction vehicles and machines using the dataset from [Kaggle](https://www.kaggle.com/datasets/kartaviychert/arh-df)

## Dataset Summary

**Construction Equipment** is a dataset for an object detection task. Possible applications of the dataset could be in the construction and surveillance industries.

The dataset consists of 318 images with **3,752** labeled objects belonging to **5** different classes including `crane`, `excavator`, `truck`, and ` tractor` and `other`.

Images in the Construction Equipment dataset have bounding box annotations. All images are labeled (i.e. with annotations). There are no pre-defined train/val/test splits in the dataset. The dataset was released in 2023.

The dataset is structured as follows:
  - **img/**: Contains all JPG images
  - **ann/**: Contains annotation files in JSON format (one per image)

You will:
1. Download and unzip the dataset from a Google Drive shared link.
2. Split the dataset into training (80%), validation (10%), and test (10%) sets.
3. Convert JSON annotations to YOLO format (.txt files) and save them in a new `labels` folder for each split.
4. Create a YOLOv5 configuration file.
5. Train a YOLOv5 model (using pretrained YOLOv5s weights) and monitor performance.
6. Evaluate the model on the test set (computing precision, recall, and mAP).
7. Run inference and display a few detected result images.

> **Note:** Update `<YOUR_GOOGLE_DRIVE_FILE_ID>` with your actual Google Drive file ID. This notebook is designed to run in Google Colab with GPU enabled.

## Step 1: Download the Dataset

We will use `gdown` to download the dataset from a shared Google Drive link. The dataset should unzip into a folder named `ARH-DF` containing two folders:

```
ARH-DF/
   ├── img/         (all .jpg images)
   └── ann/         (annotation files in JSON format)
```

In [ ]:
!pip install gdown pyyaml

### Download Dataset from Google Drive
Replace `<YOUR_GOOGLE_DRIVE_FILE_ID>` with your actual file ID.

**Dataset link**: https://drive.google.com/file/d/1gbfkO37WZl4cVxE-YhGFxqMwSDL5Xyzp/view

**Google Drive file ID**: 1gbfkO37WZl4cVxE-YhGFxqMwSDL5Xyzp

In [ ]:
import gdown

# Replace with your actual Google Drive file ID for the zipped dataset
file_id = '1gbfkO37WZl4cVxE-YhGFxqMwSDL5Xyzp'
url = f'https://drive.google.com/uc?id={file_id}'
output_zip = '/content/construction_equipment_dataset.zip'

print("Downloading dataset...")
gdown.download(url, output_zip, quiet=False)

!unzip -q /content/construction_equipment_dataset.zip -d /content/construction_equipment_dataset
print("Dataset downloaded and unzipped to /content/construction_equipment_dataset")

Downloading...
From (original): https://drive.google.com/uc?id=1gbfkO37WZl4cVxE-YhGFxqMwSDL5Xyzp
From (redirected): https://drive.google.com/uc?id=1gbfkO37WZl4cVxE-YhGFxqMwSDL5Xyzp&confirm=t&uuid=5182cff7-be01-4b33-b4c8-e5def699a97c
To: /content/construction_equipment_dataset.zip
100%|██████████| 427M/427M [00:02<00:00, 151MB/s]


replace /content/construction_equipment_dataset/img/001ebfeb-frame288.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 2: Split the Dataset and Convert Annotations

The raw dataset in `/content/construction_equipment_dataset` has two folders: `img` for images and `ann` for JSON annotation files. We'll split the images into train (80%), validation (10%), and test (10%) sets. Then, for each image, we'll convert its corresponding JSON annotation to YOLO format and save it as a `.txt` file in a new `labels` folder for that split.

In [53]:
import os, shutil, random, json

# Set seed for reproducibility
random.seed(42)

# Define raw dataset directories
base_dir = '/content/construction_equipment_dataset'
orig_images_dir = os.path.join(base_dir, 'img')
orig_ann_dir = os.path.join(base_dir, 'ann')  # JSON annotations

for filename in os.listdir(orig_ann_dir):
    if filename.endswith('.jpg.json'):
        new_filename = filename.replace('.jpg.json', '.json')
        old_path = os.path.join(orig_ann_dir, filename)
        new_path = os.path.join(orig_ann_dir, new_filename)
        os.rename(old_path, new_path)
        # print(f"Renamed {filename} to {new_filename}")

# Create directories for train, val, and test splits (each with 'images' and 'labels')
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

# List all image files (assuming .jpg)
all_images = [f for f in os.listdir(orig_images_dir) if f.endswith('.jpg')]
total_images = len(all_images)
print(f"Total images found: {total_images}")

Renamed 676153b0-frame222.jpg.json to 676153b0-frame222.json
Renamed aecb5182-frame123.jpg.json to aecb5182-frame123.json
Renamed 18ea281c-frame46.jpg.json to 18ea281c-frame46.json
Renamed 6db1a3b1-frame18.jpg.json to 6db1a3b1-frame18.json
Renamed a83e97df-frame145.jpg.json to a83e97df-frame145.json
Renamed 6989450e-frame233.jpg.json to 6989450e-frame233.json
Renamed 688ffb90-frame213.jpg.json to 688ffb90-frame213.json
Renamed cff7169c-frame170.jpg.json to cff7169c-frame170.json
Renamed 2f01e79f-frame253.jpg.json to 2f01e79f-frame253.json
Renamed 88f3d9d0-frame129.jpg.json to 88f3d9d0-frame129.json
Renamed 2c04e376-frame157.jpg.json to 2c04e376-frame157.json
Renamed aa2e495c-frame216.jpg.json to aa2e495c-frame216.json
Renamed 4eba3100-frame266.jpg.json to 4eba3100-frame266.json
Renamed 0bad1cea-frame156.jpg.json to 0bad1cea-frame156.json
Renamed 5c83fa13-frame246.jpg.json to 5c83fa13-frame246.json
Renamed be513ea8-frame74.jpg.json to be513ea8-frame74.json
Renamed 2e5d2d01-frame0.jpg.js

#### Shuffle and split images: 80% train, 10% val, 10% test

In [54]:
# Shuffle and split images: 80% train, 10% val, 10% test
random.shuffle(all_images)
train_end = int(0.8 * total_images)
val_end = int(0.9 * total_images)
train_imgs = all_images[:train_end]
val_imgs = all_images[train_end:val_end]
test_imgs = all_images[val_end:]
print(f"Dataset splits -> Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

Dataset splits -> Train: 254, Val: 32, Test: 32


### [Important!!!] Define class names (update as needed, e.g., 'vehicle', 'machine')

Before setting up the training, let's scan the JSON annotation files (in `ann`) to print out all unique class names. Update the `classes` variable accordingly.

In [55]:
import glob

# Gather all JSON files in the annotation folder
json_files = glob.glob(os.path.join(orig_ann_dir, '*.json'))

unique_classes = set()

for jf in json_files:
    with open(jf, 'r') as f:
        data = json.load(f)
    # In this dataset, annotations are under the key 'objects' and class names are stored in 'classTitle'
    for obj in data.get("objects", []):
        cls = obj.get("classTitle")
        if cls:
            unique_classes.add(cls)

print("Unique class names found in the dataset:")
for cls in sorted(unique_classes):
    print(" -", cls)

# Now update the classes variable (e.g., if the dataset contains excavator, truck)
classes = sorted(unique_classes)
print("Updated classes:", classes)

Unique class names found in the dataset:
 - crane
 - excavator
 - other
 - tractor
 - truck
Updated classes: ['crane', 'excavator', 'other', 'tractor', 'truck']


In [56]:
# Define class names (update as needed, e.g., 'vehicle', 'machine')
classes = ['crane', 'excavator', 'other', 'tractor', 'truck']

In [59]:
def convert_json_to_yolo(json_file, classes, image_file):
    """
    Expected JSON structure:
    {
        "size": {"width": ..., "height": ...},
        "objects": [
             {"classTitle": "excavator", "points": {"exterior": [[xmin, ymin], [xmax, ymax]]}, ...},
             ...
        ]
    }
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    # Get image dimensions
    if "size" in data and "width" in data["size"] and "height" in data["size"]:
        width = float(data["size"]["width"])
        height = float(data["size"]["height"])
    else:
        img = cv2.imread(image_file)
        height, width = img.shape[:2]

    yolo_lines = []
    for obj in data.get("objects", []):
        cls = obj.get("classTitle")
        if cls not in classes:
            continue
        cls_id = classes.index(cls)
        # Assume bounding box is in "points"->"exterior" with two points: top-left and bottom-right
        points = obj.get("points", {}).get("exterior", [])
        if len(points) < 2:
            continue
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height
        yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
    return yolo_lines

def copy_and_convert_files(file_list, src_images, src_ann, dest_split):
    dest_img = os.path.join(base_dir, dest_split, 'images')
    dest_lbl = os.path.join(base_dir, dest_split, 'labels')
    for fname in file_list:
        # Copy image
        shutil.copy(os.path.join(src_images, fname), dest_img)
        base_name = os.path.splitext(fname)[0]
        # For annotation, use the full filename with '.json' appended
        json_file = os.path.join(src_ann, base_name + '.json')
        image_file = os.path.join(src_images, fname)
        if os.path.exists(json_file):
            yolo_lines = convert_json_to_yolo(json_file, classes, image_file)
            if yolo_lines:
                with open(os.path.join(dest_lbl, fname + '.txt'), 'w') as f:
                    f.write("\n".join(yolo_lines))
        else:
            print(f"Warning: No annotation found for {fname}")

copy_and_convert_files(train_imgs, orig_images_dir, orig_ann_dir, 'train')
copy_and_convert_files(val_imgs, orig_images_dir, orig_ann_dir, 'val')
copy_and_convert_files(test_imgs, orig_images_dir, orig_ann_dir, 'test')
print("Dataset split and JSON annotations converted to YOLO format (labels folder).")

Dataset split and JSON annotations converted to YOLO format (labels folder).


In [60]:
for split in splits:
    images_count = len(os.listdir(os.path.join(base_dir, split, 'images')))
    labels_count = len(os.listdir(os.path.join(base_dir, split, 'labels')))
    print(f"{split.capitalize()} - Images: {images_count}, Labels: {labels_count}")

Train - Images: 254, Labels: 509
Val - Images: 32, Labels: 64
Test - Images: 32, Labels: 64


## Step 3: Create YOLOv5 Dataset Configuration

Create a YAML configuration file (`construction_equipment.yaml`) for YOLOv5 that points to your training and validation sets and lists the class names.

In [61]:
import yaml

config_data = {
    'train': os.path.join(base_dir, 'train', 'images'),
    'val': os.path.join(base_dir, 'val', 'images'),
    'test': os.path.join(base_dir, 'test', 'images'),
    'names': classes,
    'nc': len(classes)
}

with open('construction_equipment.yaml', 'w') as f:
    yaml.dump(config_data, f)
print("Created YOLOv5 dataset configuration file: construction_equipment.yaml")

Created YOLOv5 dataset configuration file: construction_equipment.yaml


## Step 4: Set Up YOLOv5 Environment

Clone the YOLOv5 repository and install dependencies. This is designed to run in Google Colab.

In [24]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print("Running in Colab?", IN_COLAB)

if IN_COLAB:
    !git clone https://github.com/ultralytics/yolov5.git
    %cd yolov5
    !pip install -r requirements.txt
else:
    print("Ensure YOLOv5 is installed locally.")

Running in Colab? True
Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.12 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.
/content/yolov5/yolov5


## Step 5: Train YOLOv5 Model

Train the model using pretrained YOLOv5s weights. Adjust epochs and batch size as needed.

In [ ]:
if IN_COLAB:
    %cd /content/yolov5
    print("Starting training...")
    !python train.py --img 640 --batch 16 --epochs 10 --data /content/construction_equipment.yaml --weights yolov5s.pt --name yolo_construction_equipment_exp
else:
    print("Run the YOLOv5 training command in your local environment.")

## Step 6: Evaluate Model Performance on Test Set

Since YOLOv5's `val.py` does not support a custom 'test' key by default, we create a temporary YAML config that uses the test set as the validation set. This allows us to compute performance metrics (precision, recall, mAP) on the test set.

In [ ]:
/content/construction_equipment_dataset/train/labels.cache
temp_config = {
    'train': os.path.join(base_dir, 'train', 'images'),  # dummy
    'val': os.path.join(base_dir, 'test', 'images'),       # use test set for evaluation
    'names': classes,
    'nc': len(classes)
}

with open('temp_test.yaml', 'w') as f:
    yaml.dump(temp_config, f)
print("Created temporary YAML config (temp_test.yaml) for test evaluation.")

if IN_COLAB:
    %cd /content/yolov5
    print("Running validation on test subset (using /content/temp_test.yaml)...")
    !python val.py --data /content/temp_test.yaml --weights runs/train/yolo_construction_equipment_exp/weights/best.pt --img 640 --conf 0.25 --half --save-json
else:
    print("Run the validation command in your local environment.")

### Load and Display Evaluation Metrics
Load the evaluation metrics from the JSON file generated by YOLOv5 and print standard metrics (precision, recall, mAP).

In [ ]:
import json

metrics_file = '/content/yolov5/runs/val/exp/results.json'
if os.path.exists(metrics_file):
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    print("\nTest Set Evaluation Metrics:")
    print("Precision:", metrics.get("precision", "N/A"))
    print("Recall:", metrics.get("recall", "N/A"))
    print("mAP@0.5:", metrics.get("mAP50", "N/A"))
    print("mAP@0.5:0.95:", metrics.get("mAP50-95", "N/A"))
else:
    print("Metrics JSON file not found. Check the validation output folder.")

## Step 7: Inference on Test Set and Visualize Detected Results

Run inference on the test set with YOLOv5 using the `--save-txt` flag (to save predicted YOLO annotations) and display the detection result images.

In [ ]:
if IN_COLAB:
    %cd /content/yolov5
    print("Running detection on test set (with --save-txt)...")
    !python detect.py --weights runs/train/yolo_construction_equipment_exp/weights/best.pt --img 640 --conf 0.25 --save-txt --source ../construction_equipment_dataset/test/img --name test_inference
else:
    print("Run the detection command in your local environment.")

### Display Detected Results
Display a few detected result images (with bounding boxes drawn) from the YOLOv5 output folder.

In [ ]:
from IPython.display import Image, display
import glob

detected_results_dir = '/content/yolov5/runs/detect/test_inference/exp'
detected_image_files = glob.glob(os.path.join(detected_results_dir, '*.jpg'))
print(f"Found {len(detected_image_files)} detected result images.")

for img_path in detected_image_files[:5]:
    display(Image(filename=img_path))

## Step 8: Tips for Improvement & Next Steps

1. Use data augmentation (e.g., Albumentations) to further increase training diversity.
2. Experiment with longer training and hyperparameter tuning (e.g., learning rate, batch size, mosaic augmentation).
3. Consider integrating the trained model with live camera feeds for real-time monitoring.
4. Analyze incorrect predictions to iterate on model improvements.

Happy Coding and Stay Safe on Site!